In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

print(os.listdir("./input"))

In [ ]:
# Data Extraction and exploration

fig_data = pd.read_csv("./input/fig_data_topicmodel.csv")

In [ ]:
fig_data.head()

In [ ]:
data_text = fig_data[['url_cleaned']]
data_text['index'] = data_text.index
documents = data_text

In [ ]:
print(len(documents))
print(documents[:5])

In [ ]:
!pip install gensim
!pip install spacy

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')



import spacy

!python -m spacy download fr
#nlp = spacy.load("fr")
#spacy.download('fr_core_web_sm')
#pacy.download('fren_core_web_sm')

In [ ]:
#tokenise words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(documents['url_cleaned']))

print(data_words[:1])

In [ ]:
#create stop words list

stop_words = set(stopwords.words('french'))

#add words that aren't in the NLTK stopwords list
new_stopwords = ['cette', 'les', 'cet']
new_stopwords_list = stop_words.union(new_stopwords)

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words and len(word) > 3] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('fr', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

In [ ]:
#key is the word and value is the number of times that word occurs in the entire corpus
dictionary = gensim.corpora.Dictionary(data_lemmatized)
print(len(dictionary))

In [ ]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 3 times
- words appearing in more than 20% of all documents
'''
dictionary.filter_extremes(no_below=3, no_above=0.20, keep_n= 100000)
print(len(dictionary))

In [ ]:
# Create Dictionary
#id2word = corpora.Dictionary(dictionary)

# Create Corpus
#bag of words
corpus = [dictionary.doc2bow(doc) for doc in data_lemmatized]

# View
print(corpus[:1])


In [ ]:
!pip install pyLDAvis
!pip install lda2vec

import pyLDAvis
import lda2vec
from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec_model import LDA2Vec

In [ ]:
n_words=len(dictionary)
# Number of documents
n_docs=len(documents)
# Number of topics to fit
n_topics=8
# Sampling temperature
temperature=1.0
# Power for neg sampling
power=0.75
# Number of unique words in the vocabulary
n_vocab=len(dictionary)
# Number of dimensions in a single word vector
n_units=300
term_frequency = np.zeros(n_vocab, dtype='int32')

In [ ]:
# Build ld2vec model
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                power=power, temperature=temperature)
model.fit(clean, components=[corpus])

In [ ]:
topics = model.prepare_topics('document_id', vocab)
prepared = pyLDAvis.prepare(topics)
pyLDAvis.display(prepared)